<a href="https://colab.research.google.com/github/ben854719/DriveSmart-An-Autonomous-Mobility-Solution/blob/main/Agentic_AI_Diagnostic_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade langchain-google-genai google-generativeai
!pip install --upgrade langchain-google-genai google-generativeai langgraph

INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
  Using cached langchain_google_genai-2.1.9-py3-none-any.whl.metadata (7.2 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-

In [27]:
from ast import Try
from IPython import get_ipython
from IPython import display
import os
from langgraph.graph import StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import SystemMessage, HumanMessage
from typing import TypedDict, List
from google.colab import userdata

#Import google colab.
Colab_Secret_key = "Ben856"

# Import API Key to function Gemini.
api_key = userdata.get("Ben856")
if not api_key:
   raise ValueError("Ben856 secret not found. Please set your API key in Colab Secrets with the same Ben856")

# Define the state of schema using TypeDict.
class LogAnalysisState(TypedDict):
  logs: List[str]
  analysis: str
  translated_text_french: str
  translated_text_spanish: str


# Initialize Gemini model.
gemini_model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=api_key)

# Create a diagnostic report of the vehicle.

def analyze_logs(state: LogAnalysisState) -> dict:
  """
  Analyze a list of diagnostic logs of the operational status of the vehicle using Gemini model.

    Args:
      state: The current state of the LanGraph workflow, containing the logs.

    Returns:
       A dictionary containing the analysis results to update the state. The key 'analysis'
       will hold a string with the details analysis, potentially included identified
       diagnosis of the diagnostic report of the status of the vehicle.
    """
  logs = state['logs']

  # Construct a more detailed prompt for the model.
  prompt_text = (
      "Analyze the following the diagnostic log on the operational status of the vehicle carefully. Your task is to identify."
      "Any potential underlying conditions, and any gaps or inconsistencies in the diagnostic process."
      "Any Highlight areas that may require further investigation of the performance of the vehicle, and suggest routine maintenance."
      "Provide a brief description on the status of the operational performance of the vehicle and indicate either in English, French, and Spanish."
      "which log entries are related.\n\n"
      "Diagnostic log:\n" + "\n".join(logs) + "\n\n"
      "Please provide your analysis in a clear and concise manner."
  )

  # Invoke the Gemini Model with the prompt wrapped in a human message.
  try:
    response = gemini_model.invoke([HumanMessage(content=prompt_text)])
    analysis_result = response.content
  except Exception as e:
    analysis_result = f"Error during analysis:  {e}"
    display.display(analysis_result)

  # Return a dictionary with the analysis content to update the state.
  return {"analysis": analysis_result}

# Translation the diagnostic log of the vehicle to French.
def translate_text_french(state: LogAnalysisState) -> dict:
  """
  Translate the text to French using Gemini model.
   _Parameters:
  text(str): The text to be translated from English.
  _Returns:
  dict: the translated text in French
  """
  text_to_translate = state['analysis']
  target_language = "French"

  prompt = f"Translate the following text From English to {target_language}: {text_to_translate}"
  response = gemini_model.invoke([HumanMessage(content=prompt)])
  return {"translated_text_french": response.content}

# Translation the diagnostic log of the vehicle French and Spanish.
def translate_text_spanish(state: LogAnalysisState) -> dict:
  """
  Translate the text to Spanish using Gemini model.
   _Parameters:
  text(str): The text to be translated from English.
  _Returns:
  dict: the translated text in Spanish
  """
  text_to_translate = state['analysis']
  target_language = "Spanish"


  prompt = f"Translate the following text From English to {target_language}: {text_to_translate}"
  response = gemini_model.invoke([HumanMessage(content=prompt)])
  return {"translated_text_spanish": response.content}


# Create LangGraph workflow.
workflow = StateGraph(state_schema=LogAnalysisState)
workflow.add_node("log_analysis", analyze_logs)
workflow.add_node("translation_french", translate_text_french)
workflow.add_node("translation_spanish", translate_text_spanish)
workflow.add_edge("log_analysis", "translation_french")
workflow.add_edge("log_analysis", "translation_spanish")

workflow.set_entry_point("log_analysis")

# Compile the workflow.
app = workflow.compile()

# Create the diagnostic log.
diagnostic_log = [
    "The model of the vehicle is a blue Mercedes E350 sedan",
    "The year of the model of the Mercedes E350 is 2025",
    "The vehicle has 1250 KM on the dashboard",
    "The Mercedes was made in Germany",
    "The Mercedes is a self-driving car as an option",
    "The Mercedes is a electric vehicle",
    "The model of the tires of the vehicle is Goodyear",
    "The size of the tires are 19 inches and all season",
    "The Mercedes has air conditioner",
    "Provide a status report to replace the cabin air filter",
    "The Mercedes has bluetooth as a remote starter to unlock the vehicle and start the vehicle",
    "The Mercedes has a Burmester 4D Surround System 17-Speakers Audio System with AI application",
    "The Artificial Intelligence provides a diagnostic report on the status of the vehicle",
    "The Artifical Intelligence used Gemini as a road assistance and provide a reminder for the next appointment for the maintenance of the vehicle",
    "The Artifical Intelligence helps to detect any anomalies in the GPS system, the engine, the brakes, the 6 airbags in the car, the tires, the status of the oil, the windshield fluid, and the heat seats",
    "The transmission of the mercedes is Automatic",
    "The light of the vehicle is LED with nightvision",
    "The vehicle hasleather seats, woodend display with a metallic blue as a color",
    "The Mercedes has a temperature detector sensor for the vehicle",
    "The Mercedes has ABS for steering wheel",
    "The Mercedes has a build in road condition detector where the Artificial Intelligence will monitor the road condition if the road is dry, wet, fog, and snowy",
    "The Artifical Intelligence would stop automatically vehicle engine if it detects a danger ahead of the car to prevent an car accident",
]

# Run the workflow.
display.display("Running log analysis workflow...")
result = app.invoke({"logs": diagnostic_log})
display.display("\nAnalysis Result:")
display.display(result['analysis'])
display.display(f"Translated text (French): {result['translated_text_french']}")
display.display(f"Translated text (Spanish): {result['translated_text_spanish']}")


# Print original and translated text
print(f"Original text: {result['analysis']}")
print(f"Translated text (French): {result['translated_text_french']}")
print(f"Translated text (Spanish): {result['translated_text_spanish']}")

'Running log analysis workflow...'

'\nAnalysis Result:'

'This diagnostic log provides more of a feature list and general vehicle information rather than a comprehensive diagnostic output. However, we can still extract valuable insights and identify areas for further investigation.\n\n---\n\n### Analysis of Diagnostic Log\n\n**1. Potential Underlying Conditions:**\n\n*   **Cabin Air Filter Status (Log Entry: "Provide a status report to replace the cabin air filter"):** This is the *only* explicit indication of a potential underlying condition or a required service. Given the very low mileage (1250 KM), a cabin air filter needing replacement is unusual unless:\n    *   It\'s a time-based replacement (e.g., annual check, and the vehicle has been sitting for a while).\n    *   The vehicle has been operated in an extremely dusty or polluted environment.\n    *   There\'s a sensor malfunction indicating premature blockage.\n\n**2. Gaps or Inconsistencies in the Diagnostic Process:**\n\n*   **Lack of Comprehensive AI Diagnostic Report (Log Entry: 

'Translated text (French): Voici la traduction du texte de l\'anglais vers le français :\n\nCe journal de diagnostic fournit davantage une liste de fonctionnalités et des informations générales sur le véhicule plutôt qu\'un rapport de diagnostic complet. Cependant, nous pouvons tout de même en extraire des informations précieuses et identifier les domaines nécessitant une enquête plus approfondie.\n\n---\n\n### Analyse du journal de diagnostic\n\n**1. Conditions sous-jacentes potentielles :**\n\n*   **État du filtre d\'habitacle (Entrée du journal : "Fournir un rapport d\'état pour remplacer le filtre d\'habitacle") :** C\'est la *seule* indication explicite d\'une condition sous-jacente potentielle ou d\'un service requis. Compte tenu du très faible kilométrage (1250 KM), un filtre d\'habitacle nécessitant un remplacement est inhabituel, à moins que :\n    *   Il s\'agisse d\'un remplacement basé sur le temps (par exemple, un contrôle annuel, et le véhicule est resté immobilisé pendan

'Translated text (Spanish): Este registro de diagnóstico proporciona más una lista de características e información general del vehículo que un resultado de diagnóstico exhaustivo. Sin embargo, aún podemos extraer información valiosa e identificar áreas para una investigación adicional.\n\n---\n\n### Análisis del Registro de Diagnóstico\n\n**1. Posibles Condiciones Subyacentes:**\n\n*   **Estado del Filtro de Aire de Cabina (Entrada del Registro: "Proporcionar un informe de estado para reemplazar el filtro de aire de la cabina"):** Esta es la *única* indicación explícita de una posible condición subyacente o un servicio requerido. Dado el muy bajo kilometraje (1250 KM), que un filtro de aire de cabina necesite reemplazo es inusual a menos que:\n    *   Sea un reemplazo basado en el tiempo (por ejemplo, una revisión anual, y el vehículo ha estado parado por un tiempo).\n    *   El vehículo haya sido operado en un ambiente extremadamente polvoriento o contaminado.\n    *   Haya un mal fu

Original text: This diagnostic log provides more of a feature list and general vehicle information rather than a comprehensive diagnostic output. However, we can still extract valuable insights and identify areas for further investigation.

---

### Analysis of Diagnostic Log

**1. Potential Underlying Conditions:**

*   **Cabin Air Filter Status (Log Entry: "Provide a status report to replace the cabin air filter"):** This is the *only* explicit indication of a potential underlying condition or a required service. Given the very low mileage (1250 KM), a cabin air filter needing replacement is unusual unless:
    *   It's a time-based replacement (e.g., annual check, and the vehicle has been sitting for a while).
    *   The vehicle has been operated in an extremely dusty or polluted environment.
    *   There's a sensor malfunction indicating premature blockage.

**2. Gaps or Inconsistencies in the Diagnostic Process:**

*   **Lack of Comprehensive AI Diagnostic Report (Log Entry: "Th